In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['API_KEY']=''

### 1. 데이터 기반 검색 (증상이 안늘어났을때의 데이터 성능 기준) + prompt

In [22]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# CSV 파일을 DataFrame으로 로드
file_path = r'C:\Users\user\langchain\study_langchain\응급처치_증상_data_pre_final.csv'
df = pd.read_csv(file_path)

# 모델 및 기타 초기화
model = ChatOpenAI(model='gpt-4')
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 1. TextSplitter를 사용하여 텍스트 분할 및 벡터 스토어 생성
split_texts = []
for _, row in df.iterrows():
    symptom = row['증상']
    treatment = row['응급처치']
    source = row['응급처치 출처']
    
    # 텍스트 결합
    full_text = f"증상: {symptom}\n응급처치: {treatment}\n출처: {source}"
    
    # 텍스트 분할
    split_texts.extend(text_splitter.split_text(full_text))

# 벡터 스토어 생성
vector_store = FAISS.from_texts(split_texts, embeddings)

# 2. 증상 입력에 따라 응급처치 정보를 검색하고 모델 응답 생성
def generate_response(symptom_input):
    # 입력된 증상을 임베딩 후 검색
    query_embedding = embeddings.embed_query(symptom_input)
    search_results = vector_store.similarity_search_by_vector(query_embedding, k=1)
    
    # 검색 결과 처리
    if search_results:
        # 가장 유사한 텍스트 가져오기
        best_match = search_results[0].page_content
    else:
        best_match = "해당 증상에 대한 응급처치 정보가 없습니다."
    
    # 모델에게 프롬프트 전달
    system_prompt = SystemMessage(
        content="사용자가 입력한 증상에 맞는 응급처치 정보를 제공하세요."
    )
    user_prompt = HumanMessage(content=best_match)
    
    # 모델 응답 생성
    response = model([system_prompt, user_prompt])
    return response.content

# 3. 예시 테스트
symptom_input = "낚시바늘에 찔렸는데 어떻게 해야 될까?"
response = generate_response(symptom_input)
print(response)

Created a chunk of size 725, which is longer than the specified 500
Created a chunk of size 725, which is longer than the specified 500
Created a chunk of size 937, which is longer than the specified 500
Created a chunk of size 502, which is longer than the specified 500


먼저, 가능하다면 부상 부위를 위로 올려 피 흐름을 최소화하세요. 그런 다음, 깨끗한 천이나 붕대로 부상 부위를 가볍게 눌러 피를 멈추게 합니다. 

비록 낚싯바늘을 직접 뽑아내려는 유혹이 있을 수 있지만, 반드시 그렇게 하지 마세요. 대신에, 낚싯바늘의 끝이 보이도록 부상 부위의 피부를 밀어내세요. 그런 다음, 바늘의 뿌리 부분을 절단하려고 시도할 수 있습니다. 

하지만, 이 작업은 전문가에게 맡기는 것이 가장 안전하며, 따라서 가능한 한 빨리 119에 전화하여 의료 도움을 청하시는 것이 가장 좋습니다. 이러한 상황에서는 항상 감염을 예방하기 위해 부상 부위를 깨끗이 유지하는 것이 중요합니다. 

이러한 치료 후에도 부상 부위에 통증, 붓기, 빨갛게 변색, 열 또는 부종이 발생하면 즉시 의사에게 보여야 합니다. 이는 감염의 징후일 수 있습니다. 

참고로, 이 정보는 응급 상황에서의 일반적인 처치 방법을 제공하며, 응급 상황에서는 항상 전문 의료인의 도움을 청해야 합니다.


## 1번모델 증상 테스트 1~ 10

In [24]:
symptom_input = "얼굴 상처 찢어짐"
response = generate_response(symptom_input)
print(response)


맞습니다. 그러나 찢어진 상처가 심한 경우에는 바로 병원에 가는 것이 가장 좋습니다. 이런 경우는 자신만으로 처치하기 어려울 수 있고, 적절한 의료치료를 받지 않으면 상처가 악화되거나 감염될 위험이 있습니다. 특히 얼굴이라면 흉터가 남을 수 있으니 신속한 처치가 필요합니다.


In [25]:
symptom_input = "머리 아프다"
response = generate_response(symptom_input)
print(response)


사용자가 입력한 증상에 맞는 응급처치 정보를 제공합니다.

1. 환자를 어둡고 조용한 방에 눕혀 안정을 취하게 하세요. 가능하다면 머리와 어깨를 약간 높여줍니다.
2. 환자의 목을 불필요하게 움직이지 않게 주의하고, 구급차가 도착할 때까지 환자 곁에서 대기해야 합니다.
3. 출혈이 있는 경우, 상처 부위를 소독된 거즈나 깨끗한 천으로 눌러서 지혈을 시도해야 합니다. 단, 머리뼈가 골절된 것 같은 경우에는 직접 부위를 누르지 마세요.
4. 환자의 호흡과 맥박을 지속적으로 확인합니다. 만약 환자가 의식을 잃거나 호흡이 멈춘 경우, 심폐소생술을 실시해야 합니다.

위 응급처치는 최대한 빠른 시간 내에 전문적인 의료진의 도움을 받아야 하는 상황입니다. 환자의 상태가 악화되지 않도록 즉시 응급실로 이송하거나 119에 신고해주세요.


In [26]:
symptom_input = "발목 염좌"
response = generate_response(symptom_input)
print(response)

사용자가 제공한 증상을 바탕으로 응급처치 정보를 제공하겠습니다.

1. 환부를 더 이상 움직이지 않게 하고, 휴식을 취하십시오. 발목 염좌 시에는 1주간 정도 쉽니다. 1~2개월 간은 과도한 운동을 피하십시오.

2. 찬물이나 얼음 주머니를 이용해 냉찜질을 하여 통증을 완화하고 부기를 줄입니다. 

3. 부어 있는 부분을 탄력 붕대로 감아 압박해주는 것이 도움이 됩니다.

4. 부어 있는 발을 심장보다 높이 들어 올려 놓으면 통증과 부기가 줄어듭니다.

5. 처음에는 냉찜질을 하고, 부기나 출혈이 줄어든 후에는 온찜질을 통해 혈액 순환을 촉진하고 영양 공급을 늘려 회복을 돕습니다.

6. 냉찜질은 1회 20분 이내, 하루에 여러 번 시행하고, 피부에 이상 징후(하얗거나 파랗게 변하는 등)가 보이면 바로 중단해야 합니다.

7. 온찜질은 손상 후 24~48시간이 지난 후에 시행하며, 운동 전에 준비 단계에서 실시하는 것이 도움이 됩니다.

위의 조치들을 취해도 증상이 계속되거나, 통증이 심해지는 경우에는 즉시 의료 기관을 방문하여 진료를 받아야 합니다.


In [27]:
symptom_input = "저체온증"
response = generate_response(symptom_input)
print(response)

사용자가 저체온증 증상을 경험하고 있다면 다음의 응급처치를 따르시기 바랍니다:

1. 만약 환자가 의식을 잃었다면, 즉시 119에 신고해야 합니다.
2. 의식이 있는 환자의 경우, 젖은 옷을 벗기고 따뜻하게 보온시켜야 합니다.
3. 환자의 열 손실을 방지하고 체온을 천천히 올려야 합니다. 이는 환자의 체온을 한 시간에 1도씩 올리는 것을 의미합니다.
4. 가볍게 저체온증에 걸린 사람은 추위나 바람을 피하게 하고 따뜻한 옷을 더 입혀야 합니다. 중등도의 증상을 보이는 사람은 우선 차가운 젖은 옷을 벗긴 뒤, 담요나 침낭 등을 사용하여 환자를 따뜻하게 해야 합니다.
5. 따뜻한 음료나 사탕과 같은 음식을 제공할 수 있습니다. 그러나, 환자가 의식이 없거나 혼란스러운 상태라면 음식을 주어서는 안 됩니다.

이러한 조치들은 응급처치이며, 가능한 한 빨리 의료진의 도움을 청해야 합니다.


In [28]:
symptom_input = "추워"
response = generate_response(symptom_input)
print(response)

사용자님이 입력한 내용은 증상이 아닌 응급처치 정보입니다. 사용자님이 응급처치 정보를 원하신다면 정확한 증상을 알려주시면 감사하겠습니다.


In [ ]:
symptom_input = "추워"
response = generate_response(symptom_input)
print(response)

변비에 대한 응급처치 및 예방법은 다음과 같습니다. 

1. 식습관 개선: 염분 섭취를 제한하고 섬유질(곡물, 과일, 채소 등)을 많이 섭취하며, 따뜻한 물로 수분 보충을 자주 하는 것이 도움이 됩니다. 음식은 천천히 꼭꼭 씹어 먹고 정량을 제때 먹는 것이 중요합니다.

2. 약물치료: 변비 상태에 따라 연변완화제, 팽변완화제, 염류하제, 자극성 하제 등을 복용할 수 있습니다. 

3. 운동: 장운동을 활성화시키기 위해 앉아 있는 것을 줄이고 자주 걷도록 합니다.

4. 배변 훈련: 변의가 있을 때 바로 화장실에 가는 것이 좋습니다. 기상 후 장 운동이 가장 활발한 시간으로 아침 식사 후 대장 반사운동을 이용하여 규칙적으로 화장실에 갑니다.

5. 배변자세: 재래식 화장실처럼 쪼그리고 앉는 자세가 항문이 더 넓게 벌어져서 대변보기가 수월합니다. 

6. 스트레스 관리: 휴식, 명상, 요가 등을 통해 스트레스를 해소하면 변비 완화에 도움이 됩니다. 

지속적인 변비 증상이 있을 경우, 전문적인 의료 기관에 찾아가 상담하고 치료를 받는 것이 좋습니다. 

출처: https://www.khna.or.kr/homecare/06_diges/gastro07.php


In [30]:
symptom_input = "머리 지끈지끈"
response = generate_response(symptom_input)
print(response)

증상: 식중독
응급처치: 
1. 환자에게 물을 충분히 마시게 하되, 작은 양씩 여러 번에 걸쳐 마시게 한다.
2. 가능하다면 구토를 유도하되, 강제로 손가락 등을 이용해 구토를 유도하면 안 된다.
3. 환자의 상태를 잘 살펴보고, 구토나 설사가 계속된다면 응급실에 즉시 방문해야 한다.
4. 환자가 충분히 휴식을 취하도록 하며, 가벼운 음식만 섭취하도록 지시한다. 

증상: 화상
응급처치: 
1. 먼저 화상부위를 차가운 물에 10~20분간 담그거나 차가운 물을 부어준다. 
2. 화상부위를 청결한 천이나 거즈로 덮는다. 
3. 화상이 심하거나 넓은 경우, 환자가 호흡곤란이나 혼란을 겪는 경우 즉시 119나 가까운 응급실에 연락한다.

증상: 심장마비
응급처치: 
1. 환자가 가슴 통증을 호소하거나 심장마비 증상을 보이면 즉시 119나 가까운 병원에 연락한다. 
2. 환자를 안정적인 위치에 눕혀준다. 
3. 환자의 의식이 없고, 호흡이 없다면 심폐소생술을 즉시 실시한다. 

※ 위 내용은 일반적인 응급처치 방법이며, 실제 상황에서는 전문적인 의료 조치가 필요할 수 있습니다. 이를 참고하여 행동하시고, 가능한 한 빠르게 의료기관의 도움을 받으시기 바랍니다.


In [31]:
symptom_input = "편두통"
response = generate_response(symptom_input)
print(response)

증상: 심한 화상
응급처치: 

1. 우선 화상부위를 차가운 물에 10~20분 정도 담가 식혀주세요. 이는 피부 손상을 최소화하고 통증을 완화하는 데 도움이 됩니다. 
2. 화상 부위를 깨끗한 천이나 밴드로 부드럽게 감싸주세요. 이는 감염을 예방하는데 도움이 됩니다. 
3. 화상이 심하거나 넓은 영역에 걸쳐있는 경우 즉시 의료 기관으로 이송해야 합니다. 
4. 통증이 심한 경우, 물을 충분히 마시고 필요하다면 통증을 완화하는 약을 복용할 수 있습니다. 

하지만 이러한 조치는 임시적인 것이며, 화상 진단은 전문적인 의료인에게 맡겨야 합니다.


In [32]:
symptom_input = "발목 심하게 아프다"
response = generate_response(symptom_input)
print(response)

4. 환자의 발작이 끝나면 안정적으로 숨을 쉴 수 있도록 고개를 뒤로 젖혀 주십시오.

5. 환자가 다시 의식을 찾을 때까지 그 곁에서 함께 있어 주십시오.

6. 만약 처음 보는 발작이거나, 발작이 계속되는 경우, 발작 후 의식이 돌아오지 않는 경우, 숨쉬기에 문제가 생기거나 감염증상이 있는 경우에는 즉시 911 또는 가까운 응긥실로 연락하여 의료 서비스를 받아야 합니다.

7. 발작 후에 다시 일어날 수 있으니, 환자를 감시하고 안정을 유지하십시오. 

8. 만약 환자가 발작 후 즉시 의식을 회복하지 못하거나, 발작이 5분 이상 지속되는 경우에는 즉시 응급실로 이동해야 합니다. 

갑작스런 발작은 무서울 수 있지만, 위와 같은 조치를 취하면 환자의 안전을 보장하고 응급 상황을 효과적으로 관리할 수 있습니다.


In [33]:
symptom_input = "위염"
response = generate_response(symptom_input)
print(response)

위염에 대한 응급처치와 관련한 정보를 잘 제공해주셨습니다. 추가로, 위염 증상이 지속되거나 심해지는 경우, 즉시 의료진의 도움을 청하시기 바랍니다. 특히, 통증이나 구토, 혈액 섞인 토나 변, 무력감, 피부나 눈의 노란 색 등의 증상이 있을 경우에는 즉시 응급실로 이동해야 합니다. 이러한 증상은 위염 외에도 위궤양, 위 출혈, 위암 등 더 심각한 질병을 나타낼 수 있습니다.


테스트 결과 
- 증상 정확률 20%, 답변 나오는 시간 평균 15초 이상

### 2. 데이터 기반 검색 (증상이 늘어났을때의 데이터 성능 기준) + prompt

In [34]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# CSV 파일을 DataFrame으로 로드
file_path = r'C:\Users\user\langchain\study_langchain\응급처치_증상_data_final.csv'
df = pd.read_csv(file_path)

# 모델 및 기타 초기화
model = ChatOpenAI(model='gpt-4')
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 1. TextSplitter를 사용하여 텍스트 분할 및 벡터 스토어 생성
split_texts = []
for _, row in df.iterrows():
    symptom = row['증상']
    treatment = row['응급처치']
    source = row['응급처치 출처']
    
    # 텍스트 결합
    full_text = f"증상: {symptom}\n응급처치: {treatment}\n출처: {source}"
    
    # 텍스트 분할
    split_texts.extend(text_splitter.split_text(full_text))

# 벡터 스토어 생성
vector_store = FAISS.from_texts(split_texts, embeddings)

# 2. 증상 입력에 따라 응급처치 정보를 검색하고 모델 응답 생성
def generate_response(symptom_input):
    # 입력된 증상을 임베딩 후 검색
    query_embedding = embeddings.embed_query(symptom_input)
    search_results = vector_store.similarity_search_by_vector(query_embedding, k=1)
    
    # 검색 결과 처리
    if search_results:
        # 가장 유사한 텍스트 가져오기
        best_match = search_results[0].page_content
    else:
        best_match = "해당 증상에 대한 응급처치 정보가 없습니다."
    
    # 모델에게 프롬프트 전달
    system_prompt = SystemMessage(
        content="사용자가 입력한 증상에 맞는 응급처치 정보를 제공하세요."
    )
    user_prompt = HumanMessage(content=best_match)
    
    # 모델 응답 생성
    response = model([system_prompt, user_prompt])
    return response.content

# 3. 예시 테스트
symptom_input = "낚시바늘에 찔렸는데 어떻게 해야 될까?"
response = generate_response(symptom_input)
print(response)

낚시 바늘에 찔렸을 때 아래와 같이 응급 처치를 할 수 있습니다.

1. 찔린 부위를 청결하게 닦아줍니다. 가능하다면 찬물에 찔린 부분을 담가두는 것이 좋습니다.
2. 바늘을 뽑을 수 있다면, 바늘의 끝부분을 찾아서 천천히 뽑아냅니다. 이때 강한 힘을 주어 서둘러 뽑으려 하면 더 큰 손상을 입힐 수 있으니 참고하세요.
3. 바늘을 뽑은 후, 찔린 부위를 물로 깨끗이 씻고 소독한 후 반창고로 감싸줍니다.
4. 바늘이 깊숙이 박혀있거나, 뽑기 힘들다면 즉시 응급실에 방문하거나 119에 전화하세요.
5. 테트라누스 예방접종을 맞은지 5년 이상 지났다면, 즉시 병원에 가서 접종을 받아야 합니다.

참고로 이러한 처리는 임시적인 처치이며, 가능한 한 빨리 전문적인 의료진의 도움을 받는 것이 중요합니다.


### 증상 테스트

In [35]:
symptom_input = "얼굴 상처 찢어짐"
response = generate_response(symptom_input)
print(response)

* 상처가 크고 심각하게 찢어진 경우, 즉시 의료 기관에 찾아가야 합니다. 이때 차량 운전이나 다른 활동을 하면서 상처를 더욱 악화시킬 수 있으므로 가능하면 다른 사람에게 도움을 청하는 것이 좋습니다.
* 상처가 너무 깊어 의료 기관에 찾아가기 전에 먼저 응급처치를 해야 하는 경우에는 상처 부위를 높이 들어 압력을 줄이고 출혈을 최소화하며, 안전핀으로 고정된 깨끗한 천이나 밴드를 사용하여 상처 부위를 압박하는 것이 좋습니다.
* 의식이 없거나, 호흡이 어렵거나, 출혈이 멈추지 않거나, 통증이 심한 경우 119나 가까운 응급실로 즉시 연락하세요.


In [36]:
symptom_input = "머리 아프다"
response = generate_response(symptom_input)
print(response)

사용자님의 증상에 따른 응급처치는 다음과 같습니다:

1. 휴식: 머리가 아플 경우 먼저 휴식을 취하는 것이 중요합니다. 조용한 곳에서 누워 휴식을 취하거나, 약간의 수면을 취하면 머리통증이 완화될 수 있습니다.

2. 약물 치료: 머리통증이 심할 경우, 일반적인 해열진통제를 복용할 수 있습니다. 하지만, 약물에 대한 알레르기나 복용에 대한 지시사항을 반드시 확인하고 복용해야 합니다.

3. 수분 섭취: 머리통증은 수분 부족으로 인해 발생할 수 있습니다. 충분한 수분을 섭취하면 머리통증이 완화될 수 있습니다.

4. 마사지: 머리, 이마, 목을 부드럽게 마사지하면 머리통증이 완화될 수 있습니다.

하지만, 이러한 조치를 취해도 머리통증이 계속되거나 심해진다면, 의사의 진료를 받는 것이 필요합니다. 특히, 머리통증이 갑자기 심하게 발생하거나, 기절, 어지럼증, 식은땀, 구토 등의 증상이 동반된다면 즉시 응급실을 방문해야 합니다.


In [37]:
symptom_input = "발목 염좌"
response = generate_response(symptom_input)
print(response)

증상: 심한 화상
응급처치: 
* 화상 부위를 식히기 위해 즉시 차가운 물에 10-20분간 담그세요. (하지만 아이스팩이나 얼음은 사용하지 마세요. 이는 화상을 더 악화시킬 수 있습니다.)
* 손상된 피부를 깨끗한 랩이나 깨끗한 천으로 덮습니다. 
* 화상이 심각한 경우나 3도 화상의 경우 (즉, 피부가 흰색, 검은색, 갈색으로 변하거나 피부 표면이 건조하거나 질겨 보이는 경우)에는 즉시 병원으로 이동해야 합니다.
* 흡입 화상(연기나 가스를 마신 경우)이 의심되면 즉시 의료진에게 연락하십시오.
* 화상 부위를 높이 놓아 붓기를 줄이고 통증을 완화시키십시오.

출처: https://www.redcross.org/get-help/how-to-prepare-for-emergencies/types-of-emergencies/burns.html


In [38]:
symptom_input = "저체온증"
response = generate_response(symptom_input)
print(response)

* 저체온증이란 체온이 정상보다 낮아져 몸의 기능이 정상적으로 작동하지 않는 상태를 의미합니다.
* 초기 증상으로는 오한, 창백한 피부, 멍한 정신상태, 판단력 저하 등이 있으며, 중증일 경우에는 차가운 배, 느린 맥박과 호흡, 졸음, 혼란 등의 증상을 보입니다.
* 의식이 없는 환자는 즉시 119에 신고해야 합니다.
* 의식이 있는 환자는 젖은 옷을 벗기고 따뜻한 환경에서 체온을 천천히 올리는 것이 중요합니다. 이를 위해 따뜻한 음료를 마시게 하거나, 따뜻한 담요로 몸을 감싸 보온하도록 합니다.
* 항상 의식 상태와 호흡을 확인하며, 상황이 악화될 경우 즉시 응급의료기관으로 이동하도록 합니다.


In [39]:
symptom_input = "추워"
response = generate_response(symptom_input)
print(response)

* 충분한 수분 섭취: 변비를 완화시키려면 하루에 최소 8잔의 물을 마셔야 합니다.
* 식이섬유를 많이 섭취: 과일, 채소, 콩, 견과류, 통곡류 등 식이섬유가 많은 음식을 섭취하면 변비를 완화시킵니다.
* 카페인이나 알코올은 피하세요: 이들은 물을 흡수하여 배설물을 단단하게 만들 수 있습니다.
* 운동: 일상적인 활동이나 운동은 장 기능을 정상화시키는데 도움이 됩니다.
* 변비가 계속되거나 심해지면 의사의 진료를 받으세요: 만약 위의 조치들이 변비를 완화시키지 못하거나 복통, 체중 감소, 혈변 등 다른 증상이 동반된다면 의사의 도움이 필요합니다. 

참고: 이 정보는 의학적 조언을 대체할 수 없으며 의사나 의료 전문가의 지시를 따르세요.


In [40]:
symptom_input = "으슬으슬"
response = generate_response(symptom_input)
print(response)

증상: 심한 두통
응급처치: 
* 편안한 환경에서 쉬세요. 어두운 조명, 조용한 환경에서 누워서 휴식을 취하시는 것이 도움이 될 수 있습니다.
* 머리를 부드럽게 마사지하시거나, 목이나 어깨 근육을 이완시키세요.
* 적당한 수분을 섭취하세요. 탈수는 두통을 악화시킬 수 있습니다.
* 비타민 B2, 마그네슘, 코엔자임 Q10 등 두통에 도움이 되는 보충제를 복용해보세요. 하지만, 이러한 보충제는 의사의 지시 없이 장기간 복용하지 마세요.
* 약물 치료가 필요한 경우 의사의 처방을 받아서 통증을 완화시키는 약을 복용하세요.
* 만약 두통이 심해지거나, 기절, 구토, 열, 목의 아픔, 어지러움, 눈의 문제, 균형 장애 등 다른 증상이 동반된다면 즉시 의사에게 연락하거나 응급실로 가야 합니다.
출처: https://www.mayoclinic.org/diseases-conditions/chronic-daily-headaches/in-depth/headaches/art-20047375


In [41]:
symptom_input = "머리 지끈지끈"
response = generate_response(symptom_input)
print(response)

사용자님의 증상을 들어보니 머리를 다치신 것 같은데요. 위에 제공된 정보는 외부 상처에 대한 처치 방법입니다. 머리 아픔의 경우 아래와 같이 대처해보세요.

* 머리 아픔이 가벼운 편이라면, 잠시 휴식을 취하거나 물을 충분히 섭취해 보세요.
* 시간이 지나도 증상이 나아지지 않거나 두통이 심해진다면, 두통약을 복용할 수 있습니다.
* 만약 두통이 계속 지속되거나, 복용한 약으로도 증상이 호전되지 않는다면 병원에 방문해서 상세한 검사를 받아보는 것이 좋습니다.
* 심한 두통, 구토, 시야장애, 의식장애 등의 증상이 동반된다면 즉시 응급실로 이동해야 합니다.
* 또한, 머리를 부딪쳤거나 추락사고 등 외부 충격이 있었다면, 뇌진탕의 가능성도 있으니 즉시 응급실로 이동해야 합니다. 

참고로, 사용자님의 증상에 대한 정확한 진단과 치료를 위해서는 반드시 의료 기관을 방문해 전문적인 의료 서비스를 받는 것이 중요합니다.


In [42]:
symptom_input = "편두통"
response = generate_response(symptom_input)
print(response)

편두통이 있을 경우 아래와 같은 응급처치를 시도해 보세요:

1. 진통제 복용: 본인이 평소에 복용하는 통증약이 있다면 복용해보세요. 하지만, 지나치게 많이 복용하지는 마세요.

2. 휴식: 어둡고 조용한 공간에서 눈을 감고 휴식을 취해보세요. 편안한 자세로 누워서 몸과 머리를 편안하게 하세요.

3. 차가운 압박: 이마나 목에 차가운 압박을 가해보세요. 차가운 물을 머리에 부은 것도 도움이 될 수 있습니다.

4. 카페인 섭취: 커피나 티 등의 카페인 함유 음료를 섭취해보세요. 카페인은 편두통을 완화시키는 효과가 있습니다.

5. 스트레칭이나 마사지: 목과 어깨의 근육을 스트레칭하거나 마사지해보세요. 이는 편두통을 완화시키는 데 도움이 될 수 있습니다.

하지만, 편두통이 장기간 지속되거나, 본인이 경험한 적 없는 심한 편두통, 편두통과 함께 발열, 구토, 시력 장애 등의 증상이 동반된다면 즉시 의료 기관을 찾아가야 합니다. 왜냐하면 이러한 증상은 더 심각한 건강 문제를 가리킬 수 있기 때문입니다.


In [43]:
symptom_input = "발목 심하게 아프다"
response = generate_response(symptom_input)
print(response)

증상: 갑자기 가슴이 아프고 숨쉬기 어렵다
응급처치: 
* 환자를 편안하게 앉힌 후, 걱정하지 않도록 안심시켜 주세요.
* 119 또는 가까운 응급실에 즉시 연라하십시오. 이러한 증상은 심장마비의 전조일 수 있습니다.
* 가능하다면, 환자에게 아스피린을 씹어 먹게 하세요. 아스피린은 혈전을 방지하고 심장마비의 피해를 줄일 수 있습니다.
* 환자가 의식을 잃으면 심폐소생술(CPR)을 실시하세요.
* 의식이 없고 정상적인 호흡이 없다면 자동심장충격기(AED)를 사용해야 합니다.

이러한 조치들은 그 자체로 치료가 아니며, 응급 상황에서 시간을 벌어주는 조치입니다. 이러한 증상이 나타나면 즉시 의료 서비스를 이용해야 합니다.


In [44]:
symptom_input = "위염"
response = generate_response(symptom_input)
print(response)

* 항산제, 위장보호제 등의 약물을 사용합니다. 이는 위의 산도를 줄이고 위벽을 보호하기 위함입니다. 의사의 처방이 필요합니다.
* 위에 부담을 줄이기 위해 여러 번에 나눠서 식사를 합니다.
* 알코올, 카페인이 든 음료, 매운 음식, 흡연 등 위를 자극하는 것들은 피해야 합니다.
* 스트레스를 줄이는 것이 중요합니다. 스트레스는 위염의 원인이 될 수 있으며, 이미 위염이 있다면 증상을 악화시킬 수 있습니다. 
* 자신의 증상이 계속 된다면, 의사의 도움을 청하도록 합니다. 위염이 장기간 지속될 경우 위암의 위험이 높아질 수 있습니다. 

환자의 상태에 따라 응급처치 방법이 달라질 수 있으니, 증상이 심하거나 지속되는 경우 의료 기관을 찾아가는 것이 좋습니다.


30% 정확률, 답변 시간 평균 15초 이상

### 3. 증상 데이터(추가본)과 1oo프로 유사도 기반 검색 모델

In [46]:
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 1. 임베딩 및 모델 초기화 (한 번만 실행)
def initialize_embeddings():
    # CSV 데이터 로드
    file_path = r'C:\Users\user\langchain\study_langchain\응급처치_증상_data_final.csv' 
    df = pd.read_csv(file_path)

    # NaN 값을 처리: '심각도2'가 NaN이면 '없음'으로 대체
    df['심각도2'] = df['심각도2'].fillna('없음')

    # 임베딩 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    # FAISS 인덱스 초기화
    split_texts = [f"증상: {row['증상']}\n심각도1: {row['심각도1']}\n심각도2: {row['심각도2']}\n응급처치: {row['응급처치']}" 
                   for _, row in df.iterrows()]
    
    # FAISS 인덱스를 텍스트로부터 생성
    index = FAISS.from_texts(split_texts, embeddings)

    print("임베딩 및 인덱스 초기화 완료.")
    return df, embeddings, index

# 모델과 인덱스를 초기화하는 함수 호출 (초기 1회 실행)
df, embeddings, index = initialize_embeddings()

# 2. 증상에 대한 응급처치 정보 및 후속 질문 생성
def generate_response_with_similarity(symptom_input):
    # 증상 전처리 (소문자화 및 불필요한 공백 제거)
    symptom_input_clean = symptom_input.lower().strip()

    # 유사한 증상들을 FAISS를 통해 검색 (증상의 다른 표현을 고려)
    query_embedding = embeddings.embed_query(symptom_input_clean)  # embed_query 사용
    search_results = index.similarity_search_by_vector(query_embedding, k=5)  # 여러 개의 검색 결과를 가져옴

    # 가장 유사한 증상들 중에서 가장 적합한 결과를 선택
    matched_row = None
    highest_similarity_score = -1  # 유사도 점수

    for result in search_results:
        matched_text = result.page_content
        symptom_in_data = matched_text.split("\n")[0].replace("증상: ", "").lower().strip()

        # 유사도 점수를 계산하는 부분
        similarity_score = result.score if hasattr(result, 'score') else 0  # 유사도 점수 직접 사용

        # 유사도 점수가 가장 높은 증상 선택
        if similarity_score > highest_similarity_score:
            highest_similarity_score = similarity_score
            matched_row = df[df['증상'].str.lower().str.strip() == symptom_in_data].iloc[0]

    if matched_row is not None:
        symptom = matched_row['증상']
        emergency_action = matched_row['응급처치']
        severity1 = matched_row['심각도1']
        severity2 = matched_row['심각도2']

        # 후속 질문에 응급처치 내용 추가 (중복 제거)
        if severity1 == '중증' and severity2 == '없음':  # 중증만 있을 때
            follow_up_question = (
                f"이 증상은 중증으로 발전할 수 있습니다. 증상이 심각하다고 판단되시면, "
                f"지역을 입력해주시면, 가까운 응급실의 주소와 전화번호를 안내해드리겠습니다. 안내해드릴까요?\n\n"
                f"응급처치는 다음과 같습니다:\n{emergency_action}\n"
            )
        elif severity1 == '경증' and severity2 == '없음':  # 경증만 있을 때
            follow_up_question = (
                f"이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:\n"
                f"{emergency_action}\n\n"
                f"증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다."
            )
        elif severity1 == '경증' and severity2 == '중증':  # 경증 + 중증 (둘 다 있을 때)
            follow_up_question = (
                f"이 증상은 경증이지만 중증으로 발전할 수 있습니다. 증상이 심각해질 수 있으므로 주의가 필요합니다. "
                f"응급처치는 다음과 같습니다:\n{emergency_action}\n\n"
                f"증상이 악화되면 즉시 병원을 방문하시길 권장합니다."
            )

        response = follow_up_question  # 응급처치 내용 포함된 후속 질문만 응답으로 설정
        return response

    else:
        return "해당 증상에 대한 응급처치 정보가 없습니다."


# 예제 호출
symptom_input = "눈이 멍들었어"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

임베딩 및 인덱스 초기화 완료.
질문: 눈이 멍들었어
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 안구를 노출시키지 않도록 젖은 거즈로 덮어줍니다.
* 상처 입은 눈뿐만 아니라 반대편의 눈도 종이컵 등으로 보호합니다.
* 환자를 눕히고 안정시킨 후 병원으로 이송합니다.
* 눈을 압박하거나 비비지 않도록 합니다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


증상 테스트

In [47]:
# 증상에 대해 응급처치를 테스트하고 싶은 경우
symptom_input = "얼굴 상처 찢어짐"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 얼굴 상처 찢어짐
답변: 이 증상은 경증이지만 중증으로 발전할 수 있습니다. 증상이 심각해질 수 있으므로 주의가 필요합니다. 응급처치는 다음과 같습니다:
* 손을 비누로 깨끗이 씻습니다.
* 출혈이 있으면 직접 압박법으로 진단을 합니다.
* 상처를 물로 깨끗이 씻어서 이물질을 제거하고 출혈 방지를 위해 조심스럽게 다룹니다.
* 마른 거즈나 천으로 물기를 닦아 냅니다.
* 의사의 지시에 따라 연고 등을 바르고 적절한 상처제품으로 드레싱하고 상처 부위를 멸균된 거즈로 덮어줍니다.

증상이 악화되면 즉시 병원을 방문하시길 권장합니다.


In [48]:
symptom_input = "머리 아프다"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 머리 아프다
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 먼 곳의 움직이지 않는 고정된 물체나 수평선을 바라본다.
* 좌석에 앉아 있는 동안 머리를 바르게 든다.
* 기도를 확보한다.
* 매운 음식과 느끼한 음식, 술, 과식은 피한다.
* 여행 30-60분 전 항히스타민제를 복용한다.
* 의사의 처방으로 멀미약을 귀 뒤에 붙인다.
* 탄수화물을 섭취하여 위를 안정시킨다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


In [49]:
symptom_input = "발목 염좌"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 발목 염좌
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 염좌 부위를 고정하고 환부를 보호합니다.
* 가능하면 휴식을 취하고 1주간은 쉬는 것이 좋습니다.
* 찬물이나 얼음주머니로 냉찜질을 하고 탄력 붕대로 압박합니다.
* 부은 다리를 심장보다 높이 들어올려 회복을 도울 수 있습니다.
* 손상 정도와 상관없이 손상된 인대는 최초부터 치료해야 재발을 방지할 수 있습니다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


In [50]:
symptom_input = "저체온증"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 저체온증
답변: 이 증상은 경증이지만 중증으로 발전할 수 있습니다. 증상이 심각해질 수 있으므로 주의가 필요합니다. 응급처치는 다음과 같습니다:
* 경한 저체온증의 초기 증상은 오한, 차고 창백한 피부, 멍함, 판단력 저하이다.
* 중한 저체온증의 후기 증상은 차가운 배, 느린 맥박 및 호흡, 마비나 졸린 상태, 착란이다.
* 응급처치는 의식 유무에 따라 구분되며, 무의식 환자의 경우 즉시 119에 신고해야 한다.
* 의식이 있는 환자는 젖은 옷을 벗겨내고 따뜻하게 보온하며, 천천히 가온시켜 열손실을 막아야 한다.

증상이 악화되면 즉시 병원을 방문하시길 권장합니다.


In [51]:
symptom_input = "추워"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 추워
답변: 이 증상은 경증이지만 중증으로 발전할 수 있습니다. 증상이 심각해질 수 있으므로 주의가 필요합니다. 응급처치는 다음과 같습니다:
* 장운동을 활성화시키기 위해 앉아 있는 것을 줄이고 자주 걷도록 합니다.
* 변의가 있을 때 바로 화장실 가기.
* 아침 식사 후 대장 반사운동을 이용하여 규칙적으로 화장실에 갑니다.
* 쪼그리고 앉는 자세나 발을 들어올리는 자세를 취하여 대변보기를 수월하게 합니다.
* 휴식, 명상, 요가 등의 동작으로 스트레스를 해소시킵니다.

증상이 악화되면 즉시 병원을 방문하시길 권장합니다.


In [52]:
symptom_input = "으슬으슬"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 으슬으슬
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 먼 곳의 움직이지 않는 고정된 물체나 수평선을 바라본다.
* 좌석에 앉아 있는 동안 머리를 바르게 든다.
* 기도를 확보한다.
* 매운 음식과 느끼한 음식, 술, 과식은 피한다.
* 여행 30-60분 전 항히스타민제를 복용한다.
* 의사의 처방으로 멀미약을 귀 뒤에 붙인다.
* 탄수화물을 섭취하여 위를 안정시킨다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


In [53]:
symptom_input = "머리 지끈지끈"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 머리 지끈지끈
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 어두운 방에 조용히 누워 아무것도 하지 않도록 합니다.
* 머리에 찬 수건을 대거나 띠로 이마를 묶어서 혈관을 압박합니다.
* 충분한 수면을 취하고, 음식을 거르지 않으며, 운동을 규칙적으로 합니다.
* 술과 카페인을 피하고, 일정한 자세를 유지하며 오래 앉지 않도록 합니다.
* 두통약은 과용하지 않습니다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


In [54]:
symptom_input = "편두통"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 편두통
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 예방치료는 두통이 자주 나타날 때나 심할 때 두통을 막기 위해 매일 약을 복용하는 것을 말합니다.
* 예방치료는 뇌 전증약, 항우울제, 고혈압약 등을 환자에 따라 선택해 사용하며, 약물 복용으로 두통이 절반 이상 줄어들었다면 성공적으로 간주합니다.
* 예방치료는 4-6개월간 유지하고, 1-2개월에 걸쳐 줄여나갑니다. 만약 두통이 다시 악화되면 약물을 증량하거나 유지합니다.
* 만성편두통 환자 중 일부는 약물치료에 효과가 없는 경우가 있으며, 이 경우 보톡스치료를 시행합니다. 보톡스는 근육 이완 효과로 통증을 억제하는데, 약 75%의 환자에서 효과를 보입니다.
* 심리적 안정을 취하는 것이 중요하며, 불안과 우울은 약물치

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


In [55]:
symptom_input = "발목 심하게 아프다"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 발목 심하게 아프다
답변: 이 증상은 경증이지만 중증으로 발전할 수 있습니다. 증상이 심각해질 수 있으므로 주의가 필요합니다. 응급처치는 다음과 같습니다:
* 환자를 편안한 자세로 유지하기 위해 베개나 쿠션을 사용합니다.
* 탈구된 부위를 고정시키기 위해 붕대나 삼각건을 사용합니다.
* 골절이나 주위 신경 및 혈관 손상 가능성을 고려하여 즉시 의료기관을 방문하거나 이송을 요청합니다.

증상이 악화되면 즉시 병원을 방문하시길 권장합니다.


In [56]:
symptom_input = "위염"
response = generate_response_with_similarity(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 위염
답변: 이 증상은 경증으로 보입니다. 응급처치는 다음과 같습니다:
* 위에 자극을 주지 않게 하기 위해 1~2일 금식하고 비경구적인 방법으로 영양과 수분을 공급합니다.
* 증세가 가라앉고 식욕이 생기면 물이나 엽차, 보리차 등을 마시고 유동식부터 시작합니다.
* 유동식에 잘 적응되면 죽이나 빵을 조금씩 먹고 자극성 음식물을 피합니다.
* 철분이 많은 식품을 섭취하고 식사시간을 규칙적으로 지키며 당질 위주로 먹고 지방을 제한합니다.

증상이 지속되거나 악화되면 가까운 병원에 방문하시길 권장합니다.


70프로 정확률, 답변 1-2초안에 답변

### bm25(20%), 유사도 (80%) 검색 모델 적용(멘토님 제안)

In [17]:
import pandas as pd
from rank_bm25 import BM25Okapi
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 1. BM25 초기화
def initialize_bm25():
    # CSV 데이터 로드
    file_path = 'C:\\Users\\user\\langchain\\study_langchain\\응급처치_증상_data_final.csv'
    df = pd.read_csv(file_path)

    # NaN 값을 처리: '심각도2'가 NaN이면 '없음'으로 대체
    df['심각도2'] = df['심각도2'].fillna('없음')

    # 띄어쓰기를 기준으로 증상 텍스트를 토크나이징
    tokenized_corpus = [symptom.split() for symptom in df['증상']]

    # BM25 모델 초기화
    bm25 = BM25Okapi(tokenized_corpus)

    print("BM25 초기화 완료.")
    return df, bm25

# 모델 초기화 (1회 실행)
df, bm25 = initialize_bm25()

# 2. 임베딩 및 모델 초기화 (한 번만 실행)
def initialize_embeddings():
    # CSV 데이터 로드
    file_path =  'C:\\Users\\user\\langchain\\study_langchain\\응급처치_증상_data_final.csv' 
    df = pd.read_csv(file_path)

    # NaN 값을 처리: '심각도2'가 NaN이면 '없음'으로 대체
    df['심각도2'] = df['심각도2'].fillna('없음')

    # 임베딩 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    # FAISS 인덱스 초기화
    split_texts = [f"증상: {row['증상']}\n심각도1: {row['심각도1']}\n심각도2: {row['심각도2']}\n응급처치: {row['응급처치']}" 
                   for _, row in df.iterrows()]
    
    # FAISS 인덱스를 텍스트로부터 생성
    index = FAISS.from_texts(split_texts, embeddings)

    print("임베딩 및 인덱스 초기화 완료.")
    return df, embeddings, index

# 모델과 인덱스를 초기화하는 함수 호출 (초기 1회 실행)
df, embeddings, index = initialize_embeddings()

# 3. 증상에 대한 응급처치 정보 및 후속 질문 생성
def generate_combined_response(symptom_input):
    # 1. BM25 기반 검색
    symptom_input_tokens = symptom_input.split()
    scores = bm25.get_scores(symptom_input_tokens)
    top_indices_bm25 = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]  # 상위 5개

    matched_bm25_indices = [i for i in top_indices_bm25 if scores[i] > 0]
    
    # BM25에서 가장 적합한 결과 선택
    best_bm25_index = matched_bm25_indices[0] if matched_bm25_indices else None
    bm25_row = df.iloc[best_bm25_index] if best_bm25_index is not None else None
    
    # 2. LLM 기반 검색 (FAISS)
    symptom_input_clean = symptom_input.lower().strip()
    query_embedding = embeddings.embed_query(symptom_input_clean)  # embed_query 사용
    search_results = index.similarity_search_by_vector(query_embedding, k=5)  # 여러 개의 검색 결과를 가져옴

    # LLM에서 가장 적합한 결과 선택
    matched_llm_row = None
    highest_similarity_score = -1

    for result in search_results:
        matched_text = result.page_content
        symptom_in_data = matched_text.split("\n")[0].replace("증상: ", "").lower().strip()
        similarity_score = result.score if hasattr(result, 'score') else 0

        if similarity_score > highest_similarity_score:
            highest_similarity_score = similarity_score
            matched_llm_row = df[df['증상'].str.lower().str.strip() == symptom_in_data].iloc[0]

    # 3. BM25와 LLM의 결과 합산 (가중치 적용)
    if bm25_row is not None and matched_llm_row is not None:
        # BM25와 LLM 결과 결합 (BM25: 20%, LLM: 80%)
        combined_score = 0.2 * scores[best_bm25_index] + 0.8 * highest_similarity_score

        # 높은 가중치를 받은 모델의 결과로 응답 생성
        if highest_similarity_score >= scores[best_bm25_index]:
            best_row = matched_llm_row
        else:
            best_row = bm25_row
    else:
        # 만약 하나라도 None이라면, 해당 모델의 결과를 사용
        best_row = bm25_row if bm25_row is not None else matched_llm_row

    # 응급처치 및 후속 질문 생성
    symptom = best_row['증상']
    emergency_action = best_row['응급처치']
    severity1 = best_row['심각도1']
    severity2 = best_row['심각도2']

    if severity1 == '중증' and severity2 == '없음':
        return (
            f"걱정이 많으시겠어요. 중증 증상일 가능성이 높아 보입니다. 먼저 마음을 차분히 하시고 제가 안내드리는 응급조치를 따라주세요.\n\n"
            f"이 증상에 대해 권장되는 응급처치는 다음과 같습니다:\n{emergency_action}\n\n"
            f"빠르게 가까운 응급실로 이동하실 것을 권장드립니다. 현재 위치를 알려주시면, 근처 응급실 정보를 제공해드릴게요."
        )
    elif severity1 == '경증' and severity2 == '없음':
        return (
            f"안심하세요. 이 증상은 경미한 것으로 보입니다.\n\n"
            f"우선적으로 권장되는 응급처치는 다음과 같습니다:\n{emergency_action}\n\n"
            f"증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. "
            f"다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. "
            f"추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!"
        )
    elif severity1 == '경증' and severity2 == '중증':
        return (
            f"현재 증상은 경미할 수 있으나, 중증으로 발전할 가능성이 있습니다. 신중히 대처하셔야 합니다.\n\n"
            f"우선 권장되는 응급처치는 다음과 같습니다:\n{emergency_action}\n\n"
            f"증상이 악화되거나 중증 증상이 나타나면 즉시 응급실을 방문하세요. "
            f"참고로, 응급실 방문 시 경증으로 분류되면 의료비 부담이 높아질 수 있습니다. "
            f"추가적인 질문이나 보상 관련 정보가 필요하시면 말씀해주세요!"
        )
    else:
        return "현재 입력된 증상에 대한 정보를 찾을 수 없습니다. 더 구체적인 증상을 말씀해주시면 도움을 드릴 수 있을 것 같아요."

# 예제 호출
symptom_input = "뱀한테 물렸어"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")


BM25 초기화 완료.
임베딩 및 인덱스 초기화 완료.
질문: 뱀한테 물렸어
답변: 현재 증상은 경미할 수 있으나, 중증으로 발전할 가능성이 있습니다. 신중히 대처하셔야 합니다.

우선 권장되는 응급처치는 다음과 같습니다:
* 환자를 안전한 곳으로 옮기고 체온 유지 후 의료기관으로 빠른 시간내 이송해야 합니다.
* 교상 부위는 심장 높이보다 아래, 기능적 자세로 고정하고 반지, 시계, 꽉 끼는 옷은 풀어주며 술, 음식, 약물 등은 투여하지 않아야 합니다.
* 물렸을 때 가장 중요한 행위는 항독소를 구비하고 적절한 치료를 제공할 수 있는 의료기관으로 가는 것입니다.
* 물린 부위에서 5∼10 cm 정도 심장 쪽에 가까운 부위를 넓은 끈이나 손수건 등으로 압박대로 묶어 피가 통할 수 있습니다.

증상이 악화되거나 중증 증상이 나타나면 즉시 응급실을 방문하세요. 참고로, 응급실 방문 시 경증으로 분류되면 의료비 부담이 높아질 수 있습니다. 추가적인 질문이나 보상 관련 정보가 필요하시면 말씀해주세요!


In [57]:
symptom_input = "얼굴 상처 찢어짐"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 얼굴 상처 찢어짐
답변: 현재 증상은 경미할 수 있으나, 중증으로 발전할 가능성이 있습니다. 신중히 대처하셔야 합니다.

우선 권장되는 응급처치는 다음과 같습니다:
* 손을 비누로 깨끗이 씻습니다.
* 출혈이 있으면 직접 압박법으로 진단을 합니다.
* 상처를 물로 깨끗이 씻어서 이물질을 제거하고 출혈 방지를 위해 조심스럽게 다룹니다.
* 마른 거즈나 천으로 물기를 닦아 냅니다.
* 의사의 지시에 따라 연고 등을 바르고 적절한 상처제품으로 드레싱하고 상처 부위를 멸균된 거즈로 덮어줍니다.

증상이 악화되거나 중증 증상이 나타나면 즉시 응급실을 방문하세요. 참고로, 응급실 방문 시 경증으로 분류되면 의료비 부담이 높아질 수 있습니다. 추가적인 질문이나 보상 관련 정보가 필요하시면 말씀해주세요!


In [58]:
symptom_input = "머리 아프다"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 머리 아프다
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 예방치료는 두통이 자주 나타날 때나 심할 때 두통을 막기 위해 매일 약을 복용하는 것을 말합니다.
* 예방치료는 뇌 전증약, 항우울제, 고혈압약 등을 환자에 따라 선택해 사용하며, 약물 복용으로 두통이 절반 이상 줄어들었다면 성공적으로 간주합니다.
* 예방치료는 4-6개월간 유지하고, 1-2개월에 걸쳐 줄여나갑니다. 만약 두통이 다시 악화되면 약물을 증량하거나 유지합니다.
* 만성편두통 환자 중 일부는 약물치료에 효과가 없는 경우가 있으며, 이 경우 보톡스치료를 시행합니다. 보톡스는 근육 이완 효과로 통증을 억제하는데, 약 75%의 환자에서 효과를 보입니다.
* 심리적 안정을 취하는 것이 중요하며, 불안과 우울은 약물치

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [59]:
symptom_input = "발목 염좌"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 발목 염좌
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 손상을 입은 부위를 고정하고 보호합니다.
* 움직이지 않고 휴식을 취하며 적어도 1주간은 쉬어야 합니다.
* 찬물이나 얼음으로 냉찜질을 하여 통증을 줄이고 근육을 완화시킵니다.
* 부은 것이 가라 앉을 때까지 탄력 붕대로 압박합니다.
* 부은 다리를 심장 위치보다 높게 올려서 회복을 도울 수 있습니다.
* 손상 정도와 상관없이 최초 손상 시 치료에 주의를 기울여야 합니다.
* 손상 후 1~3일 내에 냉찜질을 하는 것이 좋습니다.
* 냉찜질은 염증과 부종을 감소시키는 효과가 있습니다.
* 온찜질은 손상 후 24~48시간 이후에 실시하는 것이 좋습니다.
* 온찜질은 만성적인 관절과 근육의 통증을 완화하는 데 더 효과적입니다.

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [60]:
symptom_input = "저체온증"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 저체온증
답변: 현재 증상은 경미할 수 있으나, 중증으로 발전할 가능성이 있습니다. 신중히 대처하셔야 합니다.

우선 권장되는 응급처치는 다음과 같습니다:
* 경한 저체온증의 초기 증상은 오한, 차고 창백한 피부, 멍함, 판단력 저하이다.
* 중한 저체온증의 후기 증상은 차가운 배, 느린 맥박 및 호흡, 마비나 졸린 상태, 착란이다.
* 응급처치는 의식 유무에 따라 구분되며, 무의식 환자의 경우 즉시 119에 신고해야 한다.
* 의식이 있는 환자는 젖은 옷을 벗겨내고 따뜻하게 보온하며, 천천히 가온시켜 열손실을 막아야 한다.

증상이 악화되거나 중증 증상이 나타나면 즉시 응급실을 방문하세요. 참고로, 응급실 방문 시 경증으로 분류되면 의료비 부담이 높아질 수 있습니다. 추가적인 질문이나 보상 관련 정보가 필요하시면 말씀해주세요!


In [62]:
symptom_input = "으슬으슬"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 으슬으슬
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 먼 곳의 움직이지 않는 고정된 물체나 수평선을 바라본다.
* 좌석에 앉아 있는 동안 머리를 바르게 든다.
* 기도를 확보한다.
* 매운 음식과 느끼한 음식, 술, 과식은 피한다.
* 여행 30-60분 전 항히스타민제를 복용한다.
* 의사의 처방으로 멀미약을 귀 뒤에 붙인다.
* 탄수화물을 섭취하여 위를 안정시킨다.

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [63]:
symptom_input = "머리지끈지끈"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 머리지끈지끈
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 먼 곳의 움직이지 않는 고정된 물체나 수평선을 바라본다.
* 좌석에 앉아 있는 동안 머리를 바르게 든다.
* 기도를 확보한다.
* 매운 음식과 느끼한 음식, 술, 과식은 피한다.
* 여행 30-60분 전 항히스타민제를 복용한다.
* 의사의 처방으로 멀미약을 귀 뒤에 붙인다.
* 탄수화물을 섭취하여 위를 안정시킨다.

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [65]:
symptom_input = "발목 심하게 아프다"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 발목 심하게 아프다
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 손상을 입은 부위를 고정하고 보호합니다.
* 움직이지 않고 휴식을 취하며 적어도 1주간은 쉬어야 합니다.
* 찬물이나 얼음으로 냉찜질을 하여 통증을 줄이고 근육을 완화시킵니다.
* 부은 것이 가라 앉을 때까지 탄력 붕대로 압박합니다.
* 부은 다리를 심장 위치보다 높게 올려서 회복을 도울 수 있습니다.
* 손상 정도와 상관없이 최초 손상 시 치료에 주의를 기울여야 합니다.
* 손상 후 1~3일 내에 냉찜질을 하는 것이 좋습니다.
* 냉찜질은 염증과 부종을 감소시키는 효과가 있습니다.
* 온찜질은 손상 후 24~48시간 이후에 실시하는 것이 좋습니다.
* 온찜질은 만성적인 관절과 근육의 통증을 완화하는 데 더 효과적입니다.

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [66]:
symptom_input = "편두통"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 편두통
답변: 안심하세요. 이 증상은 경미한 것으로 보입니다.

우선적으로 권장되는 응급처치는 다음과 같습니다:
* 예방치료는 두통이 자주 나타날 때나 심할 때 두통을 막기 위해 매일 약을 복용하는 것을 말합니다.
* 예방치료는 뇌 전증약, 항우울제, 고혈압약 등을 환자에 따라 선택해 사용하며, 약물 복용으로 두통이 절반 이상 줄어들었다면 성공적으로 간주합니다.
* 예방치료는 4-6개월간 유지하고, 1-2개월에 걸쳐 줄여나갑니다. 만약 두통이 다시 악화되면 약물을 증량하거나 유지합니다.
* 만성편두통 환자 중 일부는 약물치료에 효과가 없는 경우가 있으며, 이 경우 보톡스치료를 시행합니다. 보톡스는 근육 이완 효과로 통증을 억제하는데, 약 75%의 환자에서 효과를 보입니다.
* 심리적 안정을 취하는 것이 중요하며, 불안과 우울은 약물치

증상이 지속되거나 악화될 경우 병원을 방문하시는 것이 좋습니다. 다만, 응급실 방문 시 경증으로 판정될 경우 높은 의료비가 발생할 수 있으니 필요성을 신중히 고려해보세요. 추가적으로 도움이 필요하시면 언제든 다시 문의해주세요!


In [20]:
symptom_input = "자동차와 부딪힐때"
response = generate_combined_response(symptom_input)
print(f"질문: {symptom_input}")
print(f"답변: {response}")

질문: 자동차와 부딪힐때
답변: 현재 증상은 경미할 수 있으나, 중증으로 발전할 가능성이 있습니다. 신중히 대처하셔야 합니다.

우선 권장되는 응급처치는 다음과 같습니다:
* 즉시 119에 신고한다.
* 머리부터 발끝까지 다발적 외상을 확인한다.
* 쇼크 증상을 확인하고 맥박, 호흡수, 혈압을 측정한다.
* 내부 출혈 가능성을 의심하며 맥박, 호흡수, 혈압이 이상적이지 않을 경우 주의가 필요하다.
* 외상 후 복통이 있는 환자는 주의 깊은 관찰이 필요하다.

증상이 악화되거나 중증 증상이 나타나면 즉시 응급실을 방문하세요. 참고로, 응급실 방문 시 경증으로 분류되면 의료비 부담이 높아질 수 있습니다. 추가적인 질문이나 보상 관련 정보가 필요하시면 말씀해주세요!


일부 증상(유사도 기반 입력)에 대한 응급처치 전달의 오류가 있습니다.
하지만 해당 증상을 엑셀 데이터 프레임에 정확히 입력한대로 입력하면 응급처치를 정확하게 전달합니다.(bm25의 영향으로 추측)
